In [8]:
import numpy as np
import numba
import params

In [48]:
species = [("preMiR", 1),
          ("preMiR_Dicer", 0),
          ("miRNA", 0),
          ("Dicer", 50)]

In [49]:
species

[('preMiR', 1), ('preMiR_Dicer', 0), ('miRNA', 0), ('Dicer', 50)]

In [66]:
def generateSystem(num_preMiR, k0, k1, k2):
    """
    Function to generate system of ODEs with n type miRNAs, all with customisable reaction rates.
    
    Args:
    num_preMiR:    int, number of pre-miRNA species
    k0:            list of floats, reaction rate for association of Dicer with pre-miRNA
    k1:            list of floats, reaction rate for disassociation of Dicer and pre-miRNA
    k2:            list of floats, reaction rate for miRNA formation
    """
    
    assert num_preMiR == len(k0), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    assert num_preMiR == len(k1), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    assert num_preMiR == len(k2), "The number of reaction rates given do not correspond with the number of ODEs in the system"
    
    basic_system = [[-k0[0], k1[0], 0], #dWT/dt
              [k0[0], -(k1[0] + k2[0]), 0], #dDicerWT/dt
              [0, 0, k2[0]]] #dmiRNA/dt
    dicer = [-k0[0], (k1[0] + k2[0]), 0] #last index:  dDicer/dt
    
    for i in range(1, num_preMiR):
        
        basic_system0 = [[-k0[0], k1[0], 0], #dWT/dt
              [k0[0], -(k1[0] + k2[0]), 0], #dDicerWT/dt
              [0, 0, k2[0]]]
        dDicer = [-k0[i], (k1[i] + k2[i]), 0]
        
        for j in range(3):
            basic_system.append([0]*len(basic_system[0]))
        for k in range(len(basic_system)):
            if k < len(basic_system) - 3:
                basic_system[k] = basic_system[k] + [0]*3 #to keep it flat
            else:
                if k == len(basic_system) - 3:
                    basic_system[k] = basic_system[k] + basic_system0[0][:3]
                elif k == len(basic_system) - 2:
                    basic_system[k] = basic_system[k] + basic_system0[1][:3]
                elif k == len(basic_system) - 1:
                    basic_system[k] = basic_system[k] + basic_system0[2][:3]
        dicer = dicer + dDicer
        
    return basic_system + [dicer]

In [79]:
def generateSpecies(num_preMiR, pMiR, dicer_pMiR, miR):
    """
    Function to generate vector of initial concentrations for all species.
    
    Args:
    num_preMiR:    int, number of pre-miRNA species
    pMiR:          list of floats, initial concentration of pre-miRNA
    dicer_pMiR:    list of floats, initial concentration Dicer and pre-miRNA complexes
    miR:           list of floats, initial concentration of miRNA
    """
    
    assert num_preMiR == len(pMiR), "The number of concentrations given do not correspond with the number of species"
    assert num_preMiR == len(dicer_pMiR), "The number of concentrations given do not correspond with the number of species"
    assert num_preMiR == len(miR), "The number of concentrations given do not correspond with the number of species"
    
    init_concs = [pMiR[0], dicer_pMiR[0], miR[0]]
    init_species = {"pMiR0": pMiR[0], "dicer_pMiR0": dicer_pMiR[0], "miR0": miR[0]}
    
    for i in range(1, num_preMiR):
        init_concs.append(pMiR[i])
        init_concs.append(dicer_pMiR[i])
        init_concs.append(miR[i])
        
        init_species["preMiR"+str(i)] = pMiR[i]
        init_species["dicer_preMiR"+str(i)] = dicer_pMiR[i]
        init_species["miR"+str(i)] = miR[i]
        
    return init_concs, init_species

In [124]:
test = generateSpecies(5, [1]*5, [0]*5, [0]*5)

In [82]:
timeit(generateSpecies(5, [1]*5, [0]*5, [0]*5))

6.16 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [90]:
test2 = generateSystem(5, [ka1]*5, [ka_1]*5, [ka2]*5)

In [92]:
timeit(generateSystem(5, [ka1]*5, [ka_1]*5, [ka2]*5))

23.3 µs ± 886 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [178]:
def test_model(t, init_values, ka, kb, kc):
    """
    Function to pass to generate, pass ODEs to ODE solver.
    
    Args:
    init_values:    Array of initial concentration of each initial concentration of every species, min 4
    ka:             Array of reaction rates for formation of pre-miR - dicer commplex
    kb:             Array of reaction rates for dissociation of pre-miR - dicer complex
    kc:             Array of reaction rates for catalysis of pre-miR to miR by dicer
    """
    
    concs_len = len(init_values) - 1
    
    assert int(concs_len/3) == len(ka), "The number of reactions do not agree with the number of reaction rates given."
    assert int(concs_len/3) == len(kb), "The number of reactions do not agree with the number of reaction rates given."
    assert int(concs_len/3) == len(kc), "The number of reactions do not agree with the number of reaction rates given."
    
    ODEs = generateSystem(int(concs_len/3), ka, kb, kc)
    
    output = [[0]]*len(ODEs)
    print(len(ODEs))
    print(len(init_values))
    print(ODEs[0] * init_values)
    
    for i in range(len(ODEs)):
        output[i].append(sum(ODEs[i] * init_values))
        print(output[i])
    
    return output

In [113]:
from scipy.integrate import solve_ivp

In [128]:
init_conc

[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 50]

In [127]:
#init_conc = test[0]
#init_conc.append(params.init_dicer)

In [179]:
res1 = solve_ivp(test_model, (0, int(params.minutes)), init_conc, args=([ka1]*5, [ka_1]*5, [ka2]*5))

16
16


ValueError: operands could not be broadcast together with shapes (15,) (16,) 